![Memories Teach](https://lh3.google.com/u/2/d/11o4JyBYhHcmz-LSt63GpOs4lfW9j39T_=w1912-h954-iv1)


`Basic to Advance in Google colab's for image processing, pattern recognition and computer vision`

[Phonepaserth SISAYKEO]

Reference: visioncolab

# Image classification using ARLNET

Reference:

J. Zhang, Y. Xie, Y. Xia and C. Shen, [Attention Residual Learning for Skin Lesion Classification](https://ieeexplore.ieee.org/document/8620285) in IEEE Transactions on Medical Imaging, vol. 38, no. 9, pp. 2092-2103, Sept. 2019, doi: 10.1109/TMI.2019.2893944.

GitHub: https://github.com/Vipermdl/ARL

In [ ]:
!git clone https://github.com/Vipermdl/ARL

Cloning into 'ARL'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 88 (delta 41), reused 21 (delta 0), pack-reused 0
Receiving objects: 100% (88/88), 79.89 KiB | 2.58 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [ ]:
%cd ARL

/content/ARL


In [ ]:
from tqdm import tqdm
from torch.utils import data
import torch.optim as optim
import torch.nn.functional as F

import pandas as pd
import os
import numpy as np
import time
import torch
from PIL import Image
import numpy as np
import glob as gb
from torchvision import transforms
from torchvision import datasets

import warnings
warnings.filterwarnings('ignore')
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision import datasets
import torchvision.transforms as transforms
import copy
import random
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 7]

from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import gc

from ARL.models.ARL import arlnet18, arlnet34, arlnet50, arlnet101, arlnet152
from ARL.models.resnet import *
from ARL.crop_transform import *

# Dataset

In [ ]:
def get_dataloaders(data_dir, input_size, batch_size=32):
    data_transforms = {
        'train': transforms.Compose([
        transforms.Resize(input_size),
        transforms.RandomRotation((-10, 10)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.7079057, 0.59156483, 0.54687315],
                             std=[0.09372108, 0.11136277, 0.12577087])
    ]),
        'val': transforms.Compose([
            transforms.Resize(input_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.7079057, 0.59156483, 0.54687315],
                             std=[0.09372108, 0.11136277, 0.12577087])
        ]),
        'test': transforms.Compose([
            transforms.Resize(input_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.7079057, 0.59156483, 0.54687315],
                             std=[0.09372108, 0.11136277, 0.12577087])
        ])
        }
    print("Initializing Datasets and Dataloaders...")

    # Create training and validation datasets
    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val', 'test']}
    #image_datasets['train'] = datasets.ImageFolder('train_patches', data_transforms['train'])
    # Create training and validation dataloaders
    dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=2) for x in ['train', 'val', 'test']}
    return dataloaders_dict


# Functions

In [ ]:
def accuracy(predictions, labels):
    #round predictions to the closest integer
    probs = torch.softmax(predictions, dim=1)
    winners = probs.argmax(dim=1)
    corrects = (winners == labels).float()
    acc = corrects.sum() / len(corrects)
    return acc

def train_epoch(model, iterator, optimizer, criterion, device):
    epoch_loss = 0
    epoch_acc  = 0
    model.train()
    for batch in iterator:

        optimizer.zero_grad()

        img, label = batch
        img = img.to(device)
        label = label.to(device)

        predictions = model(img)#.squeeze()

        loss = criterion(predictions, nn.functional.one_hot(label, num_classes=7).float())
        acc  = accuracy(predictions, label)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc  += acc.item()

    return epoch_loss/len(iterator), epoch_acc/len(iterator)

def eval_epoch(model, iterator, criterion, device):

    #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    #deactivating dropout layers
    model.eval()

    #deactivates autograd
    with torch.no_grad():
        for batch in iterator:

            img, label = batch
            img = img.to(device)
            label = label.to(device)

            #convert to 1d tensor
            predictions = model(img)#.squeeze()

            #compute loss and accuracy
            loss = criterion(predictions, nn.functional.one_hot(label, num_classes=7).float())
            acc = accuracy(predictions, label)

            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [ ]:
def testing(model, iterator, batch_size, device):

    preds = []
    lbls  = []
    predictions = torch.Tensor(len(iterator)*batch_size).to(device)
    labels      = torch.Tensor(len(iterator)*batch_size).to(device)


    #deactivating dropout layers
    model.eval()

    #deactivates autograd
    with torch.no_grad():
        for batch in iterator:

            img, label = batch
            img = img.to(device)
            label = label.to(device)

            #convert to 1d tensor
            scores = model(img)
            probs = torch.softmax(scores, dim=1)
            winners = probs.argmax(dim=1)
            preds.append(winners)
            lbls.append(label)

    torch.cat(preds, out=predictions)
    torch.cat(lbls, out=labels)

    return predictions.cpu().numpy(), labels.cpu().numpy()

def metrics(predictions, labels):
    cm = confusion_matrix(labels, predictions)
    Accuracy = np.sum(np.diag(cm))/np.sum(cm)
    Pr = np.mean(np.diag(cm) / np.sum(cm, axis = 0))
    Re = np.mean(np.diag(cm) / np.sum(cm, axis = 1))
    F1 = 2*(Pr*Re)/(Pr + Re)
    return Accuracy, Pr, Re, F1, cm

def num_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
def train(model, train_dataloader, val_dataloader, optimizer, criterion, device, num_epochs,
          saved_models_folder, saved_scores_folder, save_path, printfreq=1):

    train_loss_hist = []
    train_acc_hist  = []
    val_loss_hist   = []
    val_acc_hist    = []

    best_val_loss = float('inf')
    best_model_wts = copy.deepcopy(model.state_dict())

    t0 = time.time()
    print(' Epoch    Train Loss    Val Loss    Train Acc    Val Acc    Best    Time [min]')
    print('-'*79)

    for epoch in range(num_epochs):
        t1 = time.time()
        st = '        '

        # Training metrics
        train_loss, train_acc = train_epoch(model, train_dataloader, optimizer, criterion, device)
        train_loss_hist.append(train_loss)
        train_acc_hist.append(train_acc)

        # Validation metrics
        val_loss, val_acc     = eval_epoch(model, val_dataloader, criterion, device)
        val_loss_hist.append(val_loss)
        val_acc_hist.append(val_acc)

        # Update best model
        if val_loss < best_val_loss:
            st = '     ***'
            best_val_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(best_model_wts, os.path.join(saved_models_folder, 'best_' + save_path))

        if (epoch + 1) % printfreq == 0:
            t2 = (time.time() - t1)/60
            s = f'{epoch+1:6}{train_loss:12.4f}{val_loss:13.4f}{train_acc:13.4f}{val_acc:12.4f}{st}{t2:10.1f}'
            print(s)

        # Save current model and training information
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': val_loss,
        }, os.path.join(saved_models_folder, save_path))

        # Save the loss and accuracy values
        np.save(os.path.join(saved_scores_folder, f'{save_path}_train_loss'), train_loss_hist)
        np.save(os.path.join(saved_scores_folder, f'{save_path}_train_acc'), train_acc_hist)
        np.save(os.path.join(saved_scores_folder, f'{save_path}_val_loss'), val_loss_hist)
        np.save(os.path.join(saved_scores_folder, f'{save_path}_val_acc'), val_acc_hist)

    tfinal = (time.time() - t0)/60
    print('-'*79)
    print(f'Total time [min] for {num_epochs} Epochs: {tfinal:.1f}')
    return

# Training

In [ ]:
# skin lesions: 7 classes (old version was lunares7)
!wget https://www.dropbox.com/s/nzrvuoos7sgl5dh/exp4val.zip
!unzip -qq exp4val.zip
fpath = ''
num_classes =  7

--2023-07-19 19:22:10--  https://www.dropbox.com/s/nzrvuoos7sgl5dh/exp4val.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/nzrvuoos7sgl5dh/exp4val.zip [following]
--2023-07-19 19:22:10--  https://www.dropbox.com/s/raw/nzrvuoos7sgl5dh/exp4val.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca76f4d55fbc232463b28adbd86.dl.dropboxusercontent.com/cd/0/inline/CAKIpsso2jG0TdniXsisuwk6Tci96dSlw9ucwmDn0RTrXbq-3hccnGItBcCFCHjAOFAhEMPJkfIaekdAWvhjoP9bTkv5j7qWnfMIiNIkTVqOFneLpxeOpPS_ULOSwou0WgJ0khNrTTdoL53nhutyZMdaiKck7TMnPyCYAosWSZOcjw/file# [following]
--2023-07-19 19:22:11--  https://uca76f4d55fbc232463b28adbd86.dl.dropboxusercontent.com/cd/0/inline/CAKIpsso2jG0TdniXsisuwk6Tci96dSlw9ucwmDn0RTrXbq-3hccnGItBcCFCHjAOFAhEMPJkfIaekdAWvh

In [ ]:
data_dir = fpath
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Device: {device}')

Device: cuda


## ARLNET 50

In [ ]:
gc.collect()
torch.cuda.empty_cache()
model_name = 'arlnet50'

saved_models_folder = 'saved_models_' + model_name
saved_scores_folder = 'saved_scores_' + model_name

try:
    os.mkdir(saved_models_folder)
    os.mkdir(saved_scores_folder)
except OSError:
    print('Folders already exist!')
    pass

dataloaders_dict = get_dataloaders(data_dir, 224, batch_size=32)

Initializing Datasets and Dataloaders...


In [ ]:
# the learning rate
lr = 1e-4
weight_decay = 1e-4

# model
model = arlnet50(pretrained=True, num_classes=7)
model = model.to(device)

# loss
criterion = nn.BCEWithLogitsLoss()
criterion = criterion.to(device)

# the optimizer
optimizer = optim.SGD(model.parameters(), lr, weight_decay=weight_decay, momentum=0.9)



Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 238MB/s]


In [ ]:
num_epochs = 70
save_path = '1'

train(model, dataloaders_dict['train'], dataloaders_dict['val'], optimizer, criterion, device, num_epochs,
      saved_models_folder, saved_scores_folder, save_path, printfreq=1)

 Epoch    Train Loss    Val Loss    Train Acc    Val Acc    Best    Time [min]
-------------------------------------------------------------------------------
     1      0.4668       0.4179       0.1635      0.1860     ***       1.8
     2      0.4024       0.4045       0.2668      0.2691     ***       1.7
     3      0.3932       0.3959       0.3574      0.3376     ***       1.7
     4      0.3843       0.3867       0.4251      0.4036     ***       1.7
     5      0.3756       0.3771       0.4628      0.4479     ***       1.7
     6      0.3652       0.3675       0.4903      0.4688     ***       1.7
     7      0.3544       0.3566       0.5180      0.5071     ***       1.7
     8      0.3432       0.3457       0.5415      0.5495     ***       1.7
     9      0.3320       0.3342       0.5591      0.5674     ***       1.7
    10      0.3221       0.3253       0.5767      0.5707     ***       1.7
-------------------------------------------------------------------------------
Total time 

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)



In [ ]:
p = count_parameters(model)
print(model)
print(f'{p/1e6:8.4f}')

ARLNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## ARLNET 101

In [ ]:
gc.collect()
torch.cuda.empty_cache()
model_name = 'arlnet101'

saved_models_folder = 'saved_models_' + model_name
saved_scores_folder = 'saved_scores_' + model_name

try:
    os.mkdir(saved_models_folder)
    os.mkdir(saved_scores_folder)
except OSError:
    print('Folders already exist!')
    pass

dataloaders_dict = get_dataloaders(data_dir, 224, batch_size=32)

Initializing Datasets and Dataloaders...


In [ ]:
# the learning rate
lr = 1e-4
weight_decay = 1e-4

# model
model = arlnet101(pretrained=True, num_classes=7)
model = model.to(device)

# loss
criterion = nn.BCEWithLogitsLoss()
criterion = criterion.to(device)

# the optimizer
optimizer = optim.SGD(model.parameters(), lr, weight_decay=weight_decay, momentum=0.9)

In [ ]:
p = count_parameters(model)
print(model)
print(f'{p/1e6:8.4f}')

ARLNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
num_epochs = 70
save_path = '1'

train(model, dataloaders_dict['train'], dataloaders_dict['val'], optimizer, criterion, device, num_epochs,
      saved_models_folder, saved_scores_folder, save_path, printfreq=1)

## ARLNET 152

In [ ]:
gc.collect()
torch.cuda.empty_cache()
model_name = 'arlnet152'

saved_models_folder = 'saved_models_' + model_name
saved_scores_folder = 'saved_scores_' + model_name

try:
    os.mkdir(saved_models_folder)
    os.mkdir(saved_scores_folder)
except OSError:
    print('Folders already exist!')
    pass

dataloaders_dict = get_dataloaders(data_dir, 224, batch_size=32)

In [ ]:
# the learning rate
lr = 1e-4
weight_decay = 1e-4

# model
model = arlnet152(pretrained=True, num_classes=7)
model = model.to(device)

# loss
criterion = nn.BCEWithLogitsLoss()
criterion = criterion.to(device)

# the optimizer
optimizer = optim.SGD(model.parameters(), lr, weight_decay=weight_decay, momentum=0.9)

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/hub/checkpoints/resnet152-b121ed2d.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

In [ ]:
p = count_parameters(model)
print(model)
print(f'{p/1e6:8.4f}')

ARLNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
num_epochs = 100
save_path = '1'

train(model, dataloaders_dict['train'], dataloaders_dict['val'], optimizer, criterion, device, num_epochs,
      saved_models_folder, saved_scores_folder, save_path, printfreq=1)

# Testing

In [ ]:
data_dir = '../exp4val'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 32

## ARLNET 50

In [ ]:
gc.collect()
torch.cuda.empty_cache()

model_name = 'arlnet50'
save_path = 'best_1'

saved_models_folder = 'saved_models_' + model_name
saved_scores_folder = 'saved_scores_' + model_name

model = arlnet50(pretrained=True, num_classes=7)
model = model.to(device)

dataloaders_dict = get_dataloaders(data_dir, 224, batch_size=32)

model.load_state_dict(torch.load(os.path.join(saved_models_folder, save_path)))


Initializing Datasets and Dataloaders...


<All keys matched successfully>

In [ ]:
predictions, labels = testing(model, dataloaders_dict['test'], 32, device)
Accuracy, Pr, Re, F1, cm = metrics(predictions, labels)

print('Accuracy  = ' + str(Accuracy))
print('Precision = ' + str(Pr))
print('Recall    = ' + str(Re))
print('F1        = ' + str(F1))
print(cm)

Accuracy  = 0.5523809523809524
Precision = 0.5612959935527007
Recall    = 0.5523809523809523
F1        = 0.5568027902855137
[[ 9 12  1  4  2  0  2]
 [ 6 13  1  2  3  0  5]
 [ 4  4 10  2  2  2  6]
 [ 1  4  1 18  0  1  5]
 [ 2  1  4  2 19  1  1]
 [ 0  0  0  1  6 20  3]
 [ 1  1  0  1  0  0 27]]


## ARLNET 101

In [ ]:
gc.collect()
torch.cuda.empty_cache()

model_name = 'arlnet101'
save_path = 'best_1'

saved_models_folder = 'saved_models_' + model_name
saved_scores_folder = 'saved_scores_' + model_name

model = arlnet101(pretrained=True, num_classes=7)
model = model.to(device)

dataloaders_dict = get_dataloaders(data_dir, 224, batch_size=32)

model.load_state_dict(torch.load(os.path.join(saved_models_folder, save_path)))


In [ ]:
predictions, labels = testing(model, dataloaders_dict['test'], 32, device)
Accuracy, Pr, Re, F1, cm = metrics(predictions, labels)

print('Accuracy  = ' + str(Accuracy))
print('Precision = ' + str(Pr))
print('Recall    = ' + str(Re))
print('F1        = ' + str(F1))
print(cm)

## ARLNET 152

In [ ]:
gc.collect()
torch.cuda.empty_cache()

model_name = 'arlnet152'
save_path = 'best_1'

saved_models_folder = 'saved_models_' + model_name
saved_scores_folder = 'saved_scores_' + model_name

model = arlnet152(pretrained=True, num_classes=7)
model = model.to(device)

dataloaders_dict = get_dataloaders(data_dir, 224, batch_size=32)

model.load_state_dict(torch.load(os.path.join(saved_models_folder, save_path)))


In [ ]:
predictions, labels = testing(model, dataloaders_dict['test'], 32, device)
Accuracy, Pr, Re, F1, cm = metrics(predictions, labels)

print('Accuracy  = ' + str(Accuracy))
print('Precision = ' + str(Pr))
print('Recall    = ' + str(Re))
print('F1        = ' + str(F1))
print(cm)